In [ ]:
#%pip install cartoframes

In [149]:
from pymongo import MongoClient
from pymongo import GEOSPHERE
import pandas as pd
import sys
sys.path.append('/mnt/c/Users/cleon/Documents/CAL_CAL/ironhack/Proyectos/Headquarters-Hunt/src')
import apifunctions as af
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
import shapely.geometry
import json
from bson.json_util import dumps
import matplotlib
import seaborn as sns

In [2]:
sim_comp = pd.read_csv('../data/similarcompanies.csv')

In [16]:
client = MongoClient("localhost:27017")
db = client.get_database("Headquarters")


vanc = db.get_collection("Vancouver")
bev = db.get_collection("BeverlyHills")
irv = db.get_collection('Irvine')

In [17]:
coord = af.type_point([sim_comp['longitude'][1], sim_comp['latitude'][1]])


In [34]:
querybev = [{'$geoNear': {'near' : [sim_comp['longitude'][1], sim_comp['latitude'][1]],
                      'distanceField': 'distance',
                      'maxDistance':2,
                      'distanceMultiplier':6371,
                      'spherical':True}}]
geolocbev = bev.aggregate(querybev)
respuesta_jsonbev = json.loads(dumps(geolocbev))

In [35]:
queryvan = [{'$geoNear': {'near' : [sim_comp['longitude'][0], sim_comp['latitude'][0]],
                      'distanceField': 'distance',
                      'maxDistance':2,
                      'distanceMultiplier':6371,
                      'spherical':True}}]
geolocvan = vanc.aggregate(queryvan)
respuesta_jsonvan = json.loads(dumps(geolocvan))

In [36]:
queryirv = [{'$geoNear': {'near' : [sim_comp['longitude'][1], sim_comp['latitude'][1]],
                      'distanceField': 'distance',
                      'maxDistance':2,
                      'distanceMultiplier':6371,
                      'spherical':True}}]
geolocirv = irv.aggregate(queryirv)
respuesta_jsonirv = json.loads(dumps(geolocirv))

In [37]:
resultsbev = pd.DataFrame(respuesta_jsonbev);
resultsvanc = pd.DataFrame(respuesta_jsonvan)
resultsirv = pd.DataFrame(respuesta_jsonirv)

In [91]:
onekmvan=resultsvanc.loc[(resultsvanc['distance']<1)]
catvan = onekmvan.groupby(onekmvan['category']).mean()

In [95]:

finalvan = catvan.drop(['Pub','Salad Place','Asian Restaurant', 'Gastropub','Japanese Restaurant', 'Juice Bar', 'Mexican Restaurant', 'Print Shop', 'Restaurant', 'Seafood Restaurant', 'Steakhouse'])

In [128]:
finalvan.to_csv('../data/VancouverData.csv')

In [135]:
onekmbev=resultsbev.loc[(resultsbev['distance']<1)]
catbev = onekmbev.groupby(onekmbev['category']).mean()
finalbev = catbev.drop(['Cocktail Bar', 'Hotel', 'Hotel Bar', 'Lounge', 'Optical Shop', 'Restaurant', 'Steakhouse', 'Whisky Bar', 'Wine Bar', 'Wine Shop'])
finalbev.to_csv('../data/BeverlyHillsDatas.csv')

In [125]:
onekmirv=resultsirv.loc[(resultsirv['distance']>1)]

catirv = onekmirv.groupby(onekmirv['category']).mean()
finalirv=catirv.loc[['Airport', 'Coffee Shop','Bar','Basketball Stadium', 'Design Studio', 'Elementary School', 'Tech Startup', 'Vegetarian / Vegan Restaurant']]

In [130]:
finalirv.to_csv('../data/IrvineData.csv')

In [114]:
airport=0.5
bar=0.7
basket=0.2
school=0.6
startup=0.8
vegan= 1
starbucks=0.9
design = 0.7

In [117]:
punvanc = airport*(1/finalvan['distance'][0])+bar*(1/finalvan['distance'][1])+starbucks*(1/finalvan['distance'][2])+design*(1/finalvan['distance'][3])+startup*(1/finalvan['distance'][4])+vegan*(1/finalvan['distance'][5])

In [118]:
punvanc

7.2535507199963005

In [121]:
punbev = bar*(1/finalbev['distance'][0])+starbucks*(1/finalbev['distance'][1])+design*(1/finalbev['distance'][2])+school*(1/finalbev['distance'][3]) +startup*(1/finalbev['distance'][4]) +vegan*(1/finalbev['distance'][5])

In [123]:
punbev

7.399050578810739

In [126]:
punirv = airport*(1/finalirv['distance'][0]) + starbucks*(1/finalirv['distance'][1]) + bar*(1/finalirv['distance'][2]) + basket*(1/finalirv['distance'][3]) + design*(1/finalirv['distance'][4]) + school*(1/finalirv['distance'][5]) + startup*(1/finalirv['distance'][6]) + vegan*(1/finalirv['distance'][7])


In [127]:
punirv

0.07888895968772955

In [136]:
resultsbev.to_csv('../data/BeverlyHillsCompleteData.csv')

In [137]:
results = [punvanc, punbev, punirv]

In [143]:
finalresults = [{'Vancouver': punvanc},
                {'Beverly Hills': punbev},
                {'Irvine': punirv}
    
               ]

In [145]:
pd.DataFrame(finalresults)

,Vancouver,Beverly Hills,Irvine
0,7.253551,NaN,NaN
1,NaN,7.399051,NaN
2,NaN,NaN,0.078889
